In [1]:
from google import genai
import pandas as pd
import random
import time
from IPython.display import Markdown
import time
from tqdm import trange
GEMINI_API_KEY="***REMOVED***"
client = genai.Client(api_key=GEMINI_API_KEY)


In [2]:
import os

VIDEO_PATH = 'tesla-real-world-video-q-a/videos'

for video in os.listdir(VIDEO_PATH):
    if video.endswith('.mp4'):
        video_file = client.files.upload(file=VIDEO_PATH + video)
        print(str(video_file) + ' uploaded')


In [3]:
# CACHE THIS SO DONT HAVE TO UPLOAD EVERY TIME
video_names = ['/files/tw665rf6u2em', '/files/adot8mwqx5x8', '/files/9po1avlsm7f9', '/files/yt9a9psrka9r', '/files/p4qf8z7u09gi', '/files/tpen623uy5qr', '/files/cgwmuqbl16zr', '/files/e7hmaqm5i5ha', '/files/hklyvji2hmp7', '/files/iu1dm44rzjxb', '/files/zge9lpjs7h7q', '/files/zb88tzyjuznz', '/files/kwcwvib4tjpv', '/files/o8yib7ohyfxg', '/files/eopu01d6wp0j', '/files/ccske1io7v24', '/files/uxlhl1t9mpxc', '/files/kt7nxnixyfr9', '/files/7255bog4xq89', '/files/2nds4uxi92kd', '/files/esqldu9alume', '/files/ql30433b1vc6', '/files/9ez43wx70qdv', '/files/af34tanig9gc', '/files/pjd5wi63eb1u', '/files/bfwe4wfkjphm', '/files/ve24fszp6dgz', '/files/div8okijpkbh', '/files/splyrhu1g9xs', '/files/jztnog1jagxr', '/files/wqbstailjg60', '/files/dx7pwgwd110u', '/files/sp14h3tvduzy', '/files/n91m9yu0at27', '/files/wz80be455d0s', '/files/302jdw1o6y92', '/files/gknr0u24x64q', '/files/6qtaafyokqd0', '/files/h42kez3dh6lu', '/files/bo774opih334', '/files/f1gugtoepnxh', '/files/4b3rw572v53r', '/files/eglguei4pszy', '/files/1hiy0v2eki7q', '/files/l64min9tuwem', '/files/3wmvt82rh2nw', '/files/scdj8bb5rzc7', '/files/5tcacvfjio6t', '/files/r7f6i7w0ao7g', '/files/z8xnuwwlj74h']


In [4]:


questions = pd.read_csv('tesla-real-world-video-q-a/questions.csv')
questions['answers'] = None
questions['freq_map'] = None
questions['entropy'] = None
questions.head()

,id,question,answers,freq_map,entropy
0,1,Was ego doing a legal maneuver if its goal is ...,None,None,None
1,2,Where can ego legally park on this street? A. ...,None,None,None
2,3,What is the best description of the maneuver e...,None,None,None
3,4,Why is ego stopped? A. Judah. B. Traffic Light...,None,None,None
4,5,"What is the blinker state of the oncoming car,...",None,None,None


In [6]:
def extract_answer(text, depth = 5):
    if depth == 0:
        return "NOT FOUND"
    
    temp = client.models.generate_content(
        model='gemini-2.0-flash', contents=f'''You are an answer parser.
        Parse the correct letter choice and include the final answer surrounded by <answer></answer> XML tags. 
        
        ****  ONLY INCLUDE A SINGLE LETTER WITHIN <answer></answer> ****
        
        Prompt:\n{text}
        '''
    ).text
    
    # print(temp)
    
    try:
        cur = temp.split('<answer>')[1].split('</answer>')[0].strip()
        assert(len(cur) == 1)
        assert(cur in ['A', 'B', 'C', 'D'])
        
        if depth != 5:
            print("BAD TEXT", text)
            print("PARSED", cur)
            
        return cur
    
    except Exception as e:
        time.sleep(1)
        print("Not found in", temp)
        return extract_answer(text, depth - 1)
    
    
print(extract_answer('''Here's the breakdown:

* **Ego's Goal:** Turn right.
* **The Lane:** Marked as a right-turn lane by the roundabout sign.

The correct answer is (B). The construction barrels clearly block the right-turn lane, making it illegal for ego to use it.  Ego should wait behind the other vehicles.'''))

B


In [7]:
def parse_question(question_str):
    Q = question_str.split('A.')[0]
    
    A_str = question_str.split(Q)[1]
    
    A_str += '&.'
    
    mp = {}
    
    temp = ['A', 'B', 'C', 'D', 'E', 'F', 'G', '&']
    
    for idx in range(len(temp)-1):
        cur_choice = temp[idx]
        nxt_choice = temp[idx+1]
        
        if nxt_choice + '.' not in A_str:
            break
        
        mp[cur_choice] = A_str.split(cur_choice + '.')[1].split(nxt_choice + '.')[0]
    
    return Q, mp

print(parse_question(questions['question'][0]))

('Was ego doing a legal maneuver if its goal is to turn right at the intersection? ', {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. "})


In [11]:
def query_model(video_path, prompt):
    # print("Sending prompt:\n", prompt)
    video_file = client.files.get(name=video_path)
    result = client.models.generate_content(
                model="gemini-2.0-pro-exp-02-05",
                # gemini-2.0-flash
                contents=[
                    video_file,
                    prompt
                ]
            ).text
    # print(result)
    return result

response = query_model(video_names[0][1:], '''Was ego doing a legal maneuver if its goal is to turn right at the intersection? 

Options:

(A) It's legal as the lane is empty. 
(B) It's illegal as the right turn lane is bloacked by construction. 
(C) It's illegal as ego was cutting in other vehicles that were waiting. 
(D) It's legal but the lane ahead is way too narrow for ego to pass.
''')
print(response)

Here's the breakdown of the situation and the legality of the maneuver:

*   **The Situation:** Ego (the vehicle with the camera) is approaching an intersection where it appears they intend to turn right. There are other vehicles ahead, seemingly queued for the right turn. Ego is moving to the left of these vehicles.

*   **Analyzing the Options:**

    *   **(A) It's legal as the lane is empty:** This is not applicable here.
    *   **(B) It's illegal as the right turn lane is blocked by construction:** There's no indication of construction blocking the right-turn lane.
    *   **(C) It's illegal as ego was cutting in other vehicles that were waiting:** This is the most accurate description. By driving to the front of other vehicles, it suggests that the vehicle with the dashcam is probably jumping ahead out of turn.
    *   **(D) It's legal but the lane ahead is way too narrow for ego to pass:** The width of the lane ahead is a safety concern, not a legal one.
    
Based on the analy

In [12]:
import math

def calculate_entropy(freq_map):
    total = sum(freq_map.values())
    entropy = -sum((count / total) * math.log2(count / total) for count in freq_map.values() if count > 0)
    return entropy

# Example usage:
frequency_map = {'D': 1, 'A': 3, 'C': 1}
entropy_value = calculate_entropy(frequency_map)
print("Entropy:", entropy_value)

Entropy: 1.3709505944546687


In [ ]:
import random

num_shuffles = 1

def answer_question(video_path, question, choices):
    """
    Shuffles answer options while keeping a mapping from fixed labels (A, B, C, D)
    to the original answer keys. Queries the model multiple times, extracts the answer
    with extract_answer(), and uses majority voting over the original options.
    
    Args:
        video_path (str): Path to the video file.
        question (str): The question to ask.
        choices (dict): Dictionary mapping original letters (e.g., 'A', 'B', 'C', 'D')
                        to option text.
                        
    Returns:
        tuple: (majority_vote, vote_counts) where:
            - majority_vote: The original option letter with the most votes.
            - vote_counts: A dictionary counting votes for each original option.
    """
    votes = {}
    fixed_labels = ['A', 'B', 'C', 'D']
    
    for _ in range(num_shuffles):
        # Shuffle the original choices (list of tuples: (original_letter, option_text))
        shuffled_choices = list(choices.items())
        random.shuffle(shuffled_choices)
        
        # Build a mapping from fixed label to the original letter
        mapping = {}  # new label -> original letter
        prompt = question + '\n\nOptions:\n\n'
        for new_label, (orig_letter, option_text) in zip(fixed_labels, shuffled_choices):
            mapping[new_label] = orig_letter
            prompt += f"{new_label}. {option_text}\n"
        
        # Query the model and extract its answer using extract_answer()
        res = query_model(video_path=video_path, prompt=prompt)
        answer_new_label = extract_answer(res)
        
        # Convert the fixed label answer to the original letter
        if answer_new_label in mapping:
            orig_answer = mapping[answer_new_label]
            votes[orig_answer] = votes.get(orig_answer, 0) + 1
        else:
            votes["Invalid"] = votes.get("Invalid", 0) + 1
        time.sleep(0.5)
        
        
    # Majority voting among valid (original) options
    valid_votes = {k: v for k, v in votes.items() if k in choices}
    if valid_votes:
        majority_vote = max(valid_votes, key=valid_votes.get)
    else:
        majority_vote = "Invalid"
    
    return majority_vote, votes

# Example usage:
print(video_names[0][1:])
print('=' * 10)
print(questions.iloc[0]['question'])
print('=' * 10)
Q, C = parse_question(questions.iloc[0]['question'])
print("Parsed Question:", Q)
print("Parsed Choices:", C)

majority_answer, vote_map = answer_question(video_path=video_names[0][1:], question=Q, choices=C)
print("Majority Answer (original index):", majority_answer)
print("Vote Map (original indices):", vote_map)

files/tw665rf6u2em
Was ego doing a legal maneuver if its goal is to turn right at the intersection? A. It's legal as the lane is empty. B. It's illegal as the right turn lane is bloacked by construction. C. It's illegal as ego was cutting in other vehicles that were waiting. D. It's legal but the lane ahead is way too narrow for ego to pass.
Parsed Question: Was ego doing a legal maneuver if its goal is to turn right at the intersection? 
Parsed Choices: {'A': " It's legal as the lane is empty. ", 'B': " It's illegal as the right turn lane is bloacked by construction. ", 'C': " It's illegal as ego was cutting in other vehicles that were waiting. "}
Majority Answer (original index): C
Vote Map (original indices): {'C': 1}


In [ ]:
for question_number in trange(0, 50):
    if questions.iloc[question_number]['answers'] is not None:
        continue
    
    raw_video_path = video_names[question_number][1:]
    raw_question = questions.iloc[question_number]['question']
        
    Q, C = parse_question(raw_question)
        
    answered = False
    
    while not answered:
        try: 
            model_ans, freq_mp = answer_question(video_path=raw_video_path, question=Q, choices=C)
            
            questions.at[question_number, 'reasoning'] = model_ans
            questions.at[question_number, 'entropy'] = calculate_entropy(freq_mp)
            questions.at[question_number, 'freq_map'] = freq_mp
            
            answered = True
            
        except Exception as e:
            print(f"Error on question {question_number}: {e}")
        
        print("Rate Limit? Sleeping...")
        time.sleep(10)
        
    time.sleep(1)
    
    

  0%|          | 0/50 [00:00<?, ?it/s]

Not found in B.

BAD TEXT Based on the video, here's an analysis of the situation:

*   **What we see:** The ego vehicle approaches an intersection with a designated right-turn lane. There are a few cars ahead, seemingly waiting to turn right. The ego vehicle moves past these cars and is approaching the lane, and other cars.

*   **Analysis** It is illegal for ego to use a turning lane if other vehicle are waiting in the right turn lane.

Based on the options, here the best answer:

*   **B. It's illegal as ego was cutting in other vehicles that were waiting.**

Ego did pass other vehicles to be first in turn, in the turn lane.

PARSED B
Rate Limit? Sleeping...


  2%|▏         | 1/50 [00:15<12:38, 15.49s/it]

Not found in  B </answer>
BAD TEXT Based on the images, here's an analysis of the parking situation:

*   There's a "No Parking" sign visible on the right, near the gas station.
*   The intersection has designated crosswalks and bike lanes, making parking in those areas illegal.

The right curb is the direction Ego is travelling. There is a "No Parking" sign there. Therefore, the answer would be:

**B. No parking anywhere.**

PARSED B
Rate Limit? Sleeping...


  4%|▍         | 2/50 [00:30<12:21, 15.46s/it]

...

Rate Limit? Sleeping...


 26%|██▌       | 13/50 [03:11<09:01, 14.65s/it]

Rate Limit? Sleeping...


 28%|██▊       | 14/50 [03:25<08:45, 14.61s/it]

Rate Limit? Sleeping...


# Parse Questions

In [ ]:
letter_answers = []

for idx in trange(len(questions['reasoning'])):
    cur = questions['reasoning'][idx]
    print(cur)
    letter_answers.append(extract_answer(cur))

print(letter_answers)

  0%|          | 0/50 [00:00<?, ?it/s]

A


  2%|▏         | 1/50 [00:00<00:22,  2.19it/s]

B


  4%|▍         | 2/50 [00:00<00:21,  2.21it/s]

...

B


 98%|█████████▊| 49/50 [00:26<00:00,  2.71it/s]

A


100%|██████████| 50/50 [00:26<00:00,  1.88it/s]

['A', 'B', 'C', 'A', 'A', 'B', 'A', 'A', 'A', 'C', 'C', 'B', 'C', 'A', 'B', 'B', 'A', 'A', 'C', 'C', 'NOT FOUND', 'B', 'A', 'A', 'B', 'A', 'C', 'A', 'B', 'B', 'C', 'B', 'A', 'B', 'C', 'D', 'C', 'D', 'A', 'B', 'B', 'B', 'A', 'B', 'C', 'A', 'C', 'B', 'B', 'A']


In [ ]:
dataframe_A, dataframe_B = [], []

idx = 1
while idx <= 251:
    dataframe_A.append(idx)
    if idx-1 < len(letter_answers):
        dataframe_B.append(letter_answers[idx-1])
    else:
        dataframe_B.append('A')
    idx += 1

In [ ]:
answers = pd.DataFrame({'id': dataframe_A, 'answer': dataframe_B})


In [ ]:
answers.to_csv('baseline_answers.csv', index=False)
answers.head()

,id,answer
0,1,A
1,2,B
2,3,C
3,4,A
4,5,A
